### 1.Read the uploaded JSON file


In [0]:
df_raw = spark.read.option("multiline", True).json("/FileStore/tables/sample_data.json")
df_raw.printSchema()
df_raw.display()

root
 |-- records: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- location: struct (nullable = true)
 |    |    |    |-- city: string (nullable = true)
 |    |    |    |-- zip: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- purchases: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- item: string (nullable = true)
 |    |    |    |    |-- price: long (nullable = true)



records
"List(List(001, List(New York, 10001), Alice, List(List(Book, 20), List(Pen, 5))), List(002, List(Chicago, 60601), Bob, List(List(Notebook, 10))), List(003, List(Los Angeles, 90001), Charlie, List(List(Laptop, 900), List(Mouse, 25))), List(004, List(Houston, 77001), Diana, List(List(Bag, 50))), List(005, List(Phoenix, 85001), Ethan, List(List(Chair, 100), List(Desk, 200))), List(006, List(Philadelphia, 19101), Fiona, List(List(Monitor, 300))), List(007, List(San Antonio, 78201), George, List(List(Phone, 600), List(Charger, 20))), List(008, List(San Diego, 92101), Hannah, List(List(Keyboard, 75))), List(009, List(Dallas, 75201), Ian, List(List(Tablet, 400))), List(010, List(San Jose, 95101), Julia, List(List(Camera, 700), List(Tripod, 80))))"


### 2. Flatten JSON Fields

In [0]:
from pyspark.sql.functions import *

# Explode the records array
df_flat = df_raw.select(explode(col("records")).alias("record"))

# Flatten nested fields
df_flattened = df_flat.select(
    col("record.id").alias("id"),
    col("record.name").alias("name"),
    col("record.location.city").alias("city"),
    col("record.location.zip").alias("zip"),
    explode(col("record.purchases")).alias("purchase")
).select(
    "id", "name", "city", "zip",
    col("purchase.item").alias("item"),
    col("purchase.price").alias("price")
)

df_flattened.display()

id,name,city,zip,item,price
001,Alice,New York,10001,Book,20
001,Alice,New York,10001,Pen,5
002,Bob,Chicago,60601,Notebook,10
003,Charlie,Los Angeles,90001,Laptop,900
003,Charlie,Los Angeles,90001,Mouse,25
004,Diana,Houston,77001,Bag,50
005,Ethan,Phoenix,85001,Chair,100
005,Ethan,Phoenix,85001,Desk,200
006,Fiona,Philadelphia,19101,Monitor,300
007,George,San Antonio,78201,Phone,600


### 3. Write flattened file as external parquet table

In [0]:
# Save as parquet
df_flattened.write.mode("overwrite").parquet("/FileStore/flattened_data")

# Register as external table
spark.sql("""
CREATE TABLE IF NOT EXISTS flattened_data_table
USING PARQUET
LOCATION '/FileStore/flattened_data'
""")

# Run in a SQL cell
spark.sql("REFRESH TABLE flattened_data_table")
spark.sql("SELECT * FROM flattened_data_table").show()


+---+-------+------------+-----+--------+-----+
| id|   name|        city|  zip|    item|price|
+---+-------+------------+-----+--------+-----+
|001|  Alice|    New York|10001|    Book|   20|
|001|  Alice|    New York|10001|     Pen|    5|
|002|    Bob|     Chicago|60601|Notebook|   10|
|003|Charlie| Los Angeles|90001|  Laptop|  900|
|003|Charlie| Los Angeles|90001|   Mouse|   25|
|004|  Diana|     Houston|77001|     Bag|   50|
|005|  Ethan|     Phoenix|85001|   Chair|  100|
|005|  Ethan|     Phoenix|85001|    Desk|  200|
|006|  Fiona|Philadelphia|19101| Monitor|  300|
|007| George| San Antonio|78201|   Phone|  600|
|007| George| San Antonio|78201| Charger|   20|
|008| Hannah|   San Diego|92101|Keyboard|   75|
|009|    Ian|      Dallas|75201|  Tablet|  400|
|010|  Julia|    San Jose|95101|  Camera|  700|
|010|  Julia|    San Jose|95101|  Tripod|   80|
+---+-------+------------+-----+--------+-----+

